In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8437,2021-03-31T17:00:00,ITA,19,Sicilia,38.115697,13.362357,891,140,1031,18889,...,dati inviati aggregando il 30 e il 31 di marzo...,20.0,NaN,NaN,175354.0,0.0,2010639.0,1151407.0,ITG,ITG1
8438,2021-03-31T17:00:00,ITA,9,Toscana,43.769231,11.255889,1560,265,1825,26282,...,NaN,18.0,NaN,NaN,193401.0,1973.0,2913359.0,514371.0,ITI,ITI1
8439,2021-03-31T17:00:00,ITA,10,Umbria,43.106758,12.388247,349,60,409,4397,...,NaN,4.0,NaN,NaN,50908.0,0.0,786100.0,194771.0,ITI,ITI2
8440,2021-03-31T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,49,8,57,845,...,Si precisa che il riallineamento dei dati comu...,1.0,NaN,NaN,9100.0,198.0,81244.0,14078.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8437,2021-03-31T17:00:00,19,Sicilia,891,140,1031,18889,19920,2503,2904,...,175354,3162046,1282141.0,20.0,175354.0,0.0,2010639.0,1151407.0,ITG,ITG1
8438,2021-03-31T17:00:00,9,Toscana,1560,265,1825,26282,28107,217,1538,...,195374,3427730,1770281.0,18.0,193401.0,1973.0,2913359.0,514371.0,ITI,ITI1
8439,2021-03-31T17:00:00,10,Umbria,349,60,409,4397,4806,-239,162,...,50908,980871,337974.0,4.0,50908.0,0.0,786100.0,194771.0,ITI,ITI2
8440,2021-03-31T17:00:00,2,Valle d'Aosta,49,8,57,845,902,0,62,...,9298,95322,51514.0,1.0,9100.0,198.0,81244.0,14078.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-03-31', '2021-03-30')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-03-31T17:00:00,1,Piemonte,3873,376,4249,30810,35059,-358,2298,...,309280,3310922,1512067.0,23.0,295069.0,14211.0,2390966.0,919956.0,ITC,ITC1
1,2021-03-31T17:00:00,2,Valle d'Aosta,49,8,57,845,902,0,62,...,9298,95322,51514.0,1.0,9100.0,198.0,81244.0,14078.0,ITC,ITC2
2,2021-03-31T17:00:00,3,Lombardia,7033,863,7896,87959,95855,-667,3943,...,735484,8169813,3511485.0,54.0,702162.0,33322.0,7293704.0,876109.0,ITC,ITC4
3,2021-03-31T17:00:00,5,Veneto,1676,282,1958,36739,38697,30,2317,...,382838,6184340,1558375.0,41.0,373732.0,9106.0,4548785.0,1635555.0,ITH,ITH3
4,2021-03-31T17:00:00,6,Friuli Venezia Giulia,664,82,746,14451,15197,-176,644,...,97490,1623445,580139.0,9.0,85170.0,12320.0,1438414.0,185031.0,ITH,ITH4
5,2021-03-31T17:00:00,7,Liguria,642,68,710,6385,7095,108,383,...,89324,1235904,517132.0,2.0,89324.0,0.0,1054765.0,181139.0,ITC,ITC3
6,2021-03-31T17:00:00,8,Emilia-Romagna,3427,390,3817,68618,72435,-645,1490,...,335820,4850208,1679293.0,22.0,335592.0,228.0,3995189.0,855019.0,ITH,ITH5
7,2021-03-31T17:00:00,9,Toscana,1560,265,1825,26282,28107,217,1538,...,195374,3427730,1770281.0,18.0,193401.0,1973.0,2913359.0,514371.0,ITI,ITI1
8,2021-03-31T17:00:00,10,Umbria,349,60,409,4397,4806,-239,162,...,50908,980871,337974.0,4.0,50908.0,0.0,786100.0,194771.0,ITI,ITI2
9,2021-03-31T17:00:00,11,Marche,803,146,949,8418,9367,-17,807,...,88370,1005671,610043.0,5.0,88370.0,0.0,906993.0,98678.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-03-30T17:00:00,1,Piemonte,3855,369,4224,31193,35417,-368,1861,...,306982,3284837,1502937.0,18.0,293083.0,13899.0,2377114.0,907723.0,ITC,ITC1
1,2021-03-30T17:00:00,2,Valle d'Aosta,45,8,53,849,902,60,76,...,9236,94244,51040.0,0.0,9048.0,188.0,80933.0,13311.0,ITC,ITC2
2,2021-03-30T17:00:00,3,Lombardia,7109,862,7971,88551,96522,717,3271,...,731541,8113066,3500365.0,58.0,699071.0,32470.0,7256152.0,856914.0,ITC,ITC4
3,2021-03-30T17:00:00,5,Veneto,1629,285,1914,36753,38667,-977,1130,...,380521,6140454,1553101.0,31.0,372122.0,8399.0,4528640.0,1611814.0,ITH,ITH3
4,2021-03-30T17:00:00,6,Friuli Venezia Giulia,671,80,751,14622,15373,-133,447,...,96846,1612590,576666.0,7.0,84753.0,12093.0,1430763.0,181827.0,ITH,ITH4
5,2021-03-30T17:00:00,7,Liguria,639,70,709,6278,6987,-12,489,...,88941,1228081,514365.0,6.0,88941.0,0.0,1049735.0,178346.0,ITC,ITC3
6,2021-03-30T17:00:00,8,Emilia-Romagna,3486,392,3878,69202,73080,-810,1187,...,334343,4819016,1675028.0,22.0,334118.0,225.0,3977798.0,841218.0,ITH,ITH5
7,2021-03-30T17:00:00,9,Toscana,1542,265,1807,26083,27890,0,1180,...,193836,3398670,1760000.0,23.0,191926.0,1910.0,2896149.0,502521.0,ITI,ITI1
8,2021-03-30T17:00:00,10,Umbria,354,60,414,4631,5045,65,186,...,50746,975227,337207.0,5.0,50746.0,0.0,783248.0,191979.0,ITI,ITI2
9,2021-03-30T17:00:00,11,Marche,807,150,957,8427,9384,-88,333,...,87563,999531,606591.0,7.0,87563.0,0.0,901741.0,97790.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-03-31T17:00:00,1,Piemonte,3873,376,4249,30810,35059,-358,2298,...,3310922,1512067.0,23.0,295069.0,14211.0,2390966.0,919956.0,ITC,ITC1,4.341375e+06
1,2021-03-31T17:00:00,2,Valle d'Aosta,49,8,57,845,902,0,62,...,95322,51514.0,1.0,9100.0,198.0,81244.0,14078.0,ITC,ITC2,1.255010e+05
2,2021-03-31T17:00:00,3,Lombardia,7033,863,7896,87959,95855,-667,3943,...,8169813,3511485.0,54.0,702162.0,33322.0,7293704.0,876109.0,ITC,ITC4,1.010397e+07
3,2021-03-31T17:00:00,5,Veneto,1676,282,1958,36739,38697,30,2317,...,6184340,1558375.0,41.0,373732.0,9106.0,4548785.0,1635555.0,ITH,ITH3,4.907704e+06
4,2021-03-31T17:00:00,6,Friuli Venezia Giulia,664,82,746,14451,15197,-176,644,...,1623445,580139.0,9.0,85170.0,12320.0,1438414.0,185031.0,ITH,ITH4,1.211357e+06
5,2021-03-31T17:00:00,7,Liguria,642,68,710,6385,7095,108,383,...,1235904,517132.0,2.0,89324.0,0.0,1054765.0,181139.0,ITC,ITC3,1.543127e+06
6,2021-03-31T17:00:00,8,Emilia-Romagna,3427,390,3817,68618,72435,-645,1490,...,4850208,1679293.0,22.0,335592.0,228.0,3995189.0,855019.0,ITH,ITH5,4.467118e+06
7,2021-03-31T17:00:00,9,Toscana,1560,265,1825,26282,28107,217,1538,...,3427730,1770281.0,18.0,193401.0,1973.0,2913359.0,514371.0,ITI,ITI1,3.722729e+06
8,2021-03-31T17:00:00,10,Umbria,349,60,409,4397,4806,-239,162,...,980871,337974.0,4.0,50908.0,0.0,786100.0,194771.0,ITI,ITI2,8.802850e+05
9,2021-03-31T17:00:00,11,Marche,803,146,949,8418,9367,-17,807,...,1005671,610043.0,5.0,88370.0,0.0,906993.0,98678.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-03-31T17:00:00,1,Piemonte,3873,376,4249,30810,35059,-358,2298,...,919956.0,ITC,ITC1,4.341375e+06,Piemonte,3873,5824,376,628,99
1,2021-03-31T17:00:00,2,Valle d'Aosta,49,8,57,845,902,0,62,...,14078.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,49,227,8,30,6
2,2021-03-31T17:00:00,3,Lombardia,7033,863,7896,87959,95855,-667,3943,...,876109.0,ITC,ITC4,1.010397e+07,Lombardia,7033,13692,863,1416,114
3,2021-03-31T17:00:00,5,Veneto,1676,282,1958,36739,38697,30,2317,...,1635555.0,ITH,ITH3,4.907704e+06,Veneto,1676,6000,282,1000,0
4,2021-03-31T17:00:00,6,Friuli Venezia Giulia,664,82,746,14451,15197,-176,644,...,185031.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,664,1277,82,175,0
5,2021-03-31T17:00:00,7,Liguria,642,68,710,6385,7095,108,383,...,181139.0,ITC,ITC3,1.543127e+06,Liguria,642,1801,68,226,12
6,2021-03-31T17:00:00,8,Emilia-Romagna,3427,390,3817,68618,72435,-645,1490,...,855019.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,3427,6515,390,760,0
7,2021-03-31T17:00:00,9,Toscana,1560,265,1825,26282,28107,217,1538,...,514371.0,ITI,ITI1,3.722729e+06,Toscana,1560,5033,265,622,17
8,2021-03-31T17:00:00,10,Umbria,349,60,409,4397,4806,-239,162,...,194771.0,ITI,ITI2,8.802850e+05,Umbria,349,860,60,139,2
9,2021-03-31T17:00:00,11,Marche,803,146,949,8418,9367,-17,807,...,98678.0,ITI,ITI3,1.518400e+06,Marche,803,1356,146,250,16


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-03-31T17:00:00,3,Lombardia,7033,863,7896,87959,95855,-667,3943,...,60.95,-592,672,56747,11120.0,100,4510,6.95,0.03902,7.28
1,2021-03-31T17:00:00,19,Sicilia,891,140,1031,18889,19920,2503,2904,...,16.79,2481,2904,14623,2904.0,21,380,19.86,0.05845,3.53
2,2021-03-31T17:00:00,5,Veneto,1676,282,1958,36739,38697,30,2317,...,28.20,-14,1187,43886,5274.0,35,2252,5.28,0.04721,7.80
3,2021-03-31T17:00:00,1,Piemonte,3873,376,4249,30810,35059,-358,2298,...,59.87,-383,437,26085,9130.0,39,2617,8.81,0.05293,7.12
4,2021-03-31T17:00:00,15,Campania,1587,160,1747,91370,93117,-1088,2016,...,25.81,-1085,443,23206,11496.0,38,3066,8.69,0.03484,5.83
5,2021-03-31T17:00:00,16,Puglia,1840,260,2100,44757,46857,81,1962,...,44.83,108,435,13847,5377.0,51,1830,14.17,0.04895,4.82
6,2021-03-31T17:00:00,12,Lazio,3044,371,3415,47636,51051,-47,1800,...,39.34,-54,207,34478,19144.0,38,1809,5.22,0.03069,4.87
7,2021-03-31T17:00:00,9,Toscana,1560,265,1825,26282,28107,217,1538,...,42.60,199,358,29060,10281.0,19,1302,5.29,0.04131,5.25
8,2021-03-31T17:00:00,8,Emilia-Romagna,3427,390,3817,68618,72435,-645,1490,...,51.32,-584,303,31192,4265.0,58,2064,4.78,0.03335,7.52
9,2021-03-31T17:00:00,11,Marche,803,146,949,8418,9367,-17,807,...,58.40,-9,474,6140,3452.0,10,814,13.14,0.05315,5.82


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,Lombardia,863,553,1416,60.95
1,Piemonte,376,252,628,59.87
2,Marche,146,104,250,58.40
3,P.A. Trento,51,39,90,56.67
4,Emilia-Romagna,390,370,760,51.32
5,Friuli Venezia Giulia,82,93,175,46.86
6,Puglia,260,320,580,44.83
7,Umbria,60,79,139,43.17
8,Toscana,265,357,622,42.60
9,Molise,16,23,39,41.03


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))